In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

# generate the molecular Hamiltonian for H3+
symbols = ["O", "O","N", "C", "C", "H","H","H","H","H"]
geometry = np.array([-2.4677,  0.9906,  -0.0151,
-0.7252, -1.9381, -0.0074,
3.4422, -0.1218,  -0.1347,
1.0912,  1.1800,  0.0118,
1.8146,  1.8633,  -1.1974,
3.9811,  0.1337, -0.4000,
4.2389,  0.3622,  0.5319,
2.9907, -0.9121,  -0.0990,
2.3309,  2.7976, -1.3343,
0.6485,  1.0957, -1.7221])

molecule = qchem.Molecule(symbols, geometry, charge=0)

H2mol, qubits = qchem.molecular_hamiltonian(molecule)
wires = list(range(qubits))
dev = qml.device("default.qubit", wires=qubits)

# create all possible excitations in H3+
singles, doubles = qchem.excitations(2, qubits)
excitations = singles + doubles

In [ ]:
@qml.qnode(dev)
def circuit_VQE(theta, initial_state):
    qml.StatePrep(initial_state, wires=wires)
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(theta[i], wires=excitation)
        else:
            qml.SingleExcitation(theta[i], wires=excitation)
    return qml.expval(H2mol)

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = np.array(np.zeros(len(excitations)), requires_grad=True)
delta_E, iteration = 10, 0
results_hf = []

# run the VQE optimization loop until convergence threshold is reached
while abs(delta_E) > 1e-5:
    theta, prev_energy = opt.step_and_cost(circuit_VQE, theta, initial_state=wf_hf)
    new_energy = circuit_VQE(theta, initial_state=wf_hf)
    delta_E = new_energy - prev_energy
    results_hf.append(new_energy)
    if len(results_hf) % 5 == 0:
        print(f"Step = {len(results_hf)},  Energy = {new_energy:.6f} Ha")
print(f"Starting with HF state took {len(results_hf)} iterations until convergence.")

In [ ]:
theta = np.array(np.zeros(len(excitations)), requires_grad=True)
delta_E, iteration = 10, 0
results_cisd = []

while abs(delta_E) > 1e-5:
    theta, prev_energy = opt.step_and_cost(circuit_VQE, theta, initial_state=wf_cisd)
    new_energy = circuit_VQE(theta, initial_state=wf_cisd)
    delta_E = new_energy - prev_energy
    results_cisd.append(new_energy)
    if len(results_cisd) % 5 == 0:
        print(f"Step = {len(results_cisd)},  Energy = {new_energy:.6f} Ha")
print(f"Starting with CISD state took {len(results_cisd)} iterations until convergence.")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(range(len(results_hf)), results_hf, color="r", marker="o", label="wf_hf")
ax.plot(range(len(results_cisd)), results_cisd, color="b", marker="o", label="wf_cisd")
ax.legend(fontsize=16)
ax.tick_params(axis="both", labelsize=16)
ax.set_xlabel("Iteration", fontsize=20)
ax.set_ylabel("Energy, Ha", fontsize=20)
plt.tight_layout()
plt.show()